# Project 01: Pandas Basics - Data Analysis and Manipulation

## Dataset
The project uses `proj1_ex01.csv` which contains sample data for demonstrating pandas functionality.

## 1. Import Required Libraries

In [ ]:
import pandas as pd
import json
import re
from pathlib import Path

DATA_PATH = Path('data')
OUTPUT_PATH = Path('output')
OUTPUT_PATH.mkdir(exist_ok=True)

## 2. Load and Explore Data

In [ ]:
df = pd.read_csv(DATA_PATH / "proj1_ex01.csv")

print(f"Dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

Dataset shape: (7, 8)
Columns: ['First column', 'two', 'three', 'SOME;NAME', 'five', 'What is this even?', 'seven', 'eight']


In [3]:
# Display first 10 rows
df.head(10)

,First column,two,three,SOME;NAME,five,What is this even?,seven,eight
0,0.348554,-0.14509562920877161,-0.012336991474672475,9,red,good,quarrelsome,2016-05-26 09:33:42
1,-1.493853,0.12436946488785079,1.4611100361038865,4,red,bad,doctor,2016-12-03 18:55:52
2,-0.325891,NaN,-0.42191202598625566,2,red,average,large,2016-05-15 11:49:26
3,-0.506596,0.3991147675939107,-0.26502607502330217,5,green,average,muddled,2015-01-30 22:33:29
4,NaN,-0.6913144223047157,-0.26502607502330217,2,blue,good,coordinated,2015-11-20 00:15:35
5,0.527112,2.584347847701393,-0.26502607502330217XYZ,2,blue,good,separate,2017-11-17 09:58:54
6,-1.555290,unknown,-0.7732649697439955,5,green,bad,bright,2017-05-01 10:32:41


In [4]:
# Basic dataset information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   First column        6 non-null      float64
 1   two                 6 non-null      object 
 2   three               7 non-null      object 
 3   SOME;NAME           7 non-null      int64  
 4   five                7 non-null      object 
 5   What is this even?  7 non-null      object 
 6   seven               7 non-null      object 
 7   eight               7 non-null      object 
dtypes: float64(1), int64(1), object(6)
memory usage: 576.0+ bytes


## 3. Column Analysis and Metadata Generation

In [ ]:
def analyze_columns(dataframe):
    columns_info = []
    
    for col in dataframe.columns:
        missing_percent = dataframe[col].isnull().mean()
        
        # Determine data type
        if pd.api.types.is_integer_dtype(dataframe[col]):
            data_type = "integer"
        elif pd.api.types.is_float_dtype(dataframe[col]):
            data_type = "float"
        elif pd.api.types.is_datetime64_any_dtype(dataframe[col]):
            data_type = "datetime"
        elif pd.api.types.is_bool_dtype(dataframe[col]):
            data_type = "boolean"
        else:
            data_type = "object"
        
        column_info = {
            "name": col,
            "missing_percent": round(missing_percent * 100, 2),
            "data_type": data_type,
            "unique_values": dataframe[col].nunique(),
            "memory_usage": dataframe[col].memory_usage(deep=True)
        }
        columns_info.append(column_info)
    
    return columns_info

# Analyze columns
columns_metadata = analyze_columns(df)

# Display the analysis
for info in columns_metadata:
    print(f"Column: {info['name']}")
    print(f"  Type: {info['data_type']}")
    print(f"  Missing: {info['missing_percent']}%")
    print(f"  Unique values: {info['unique_values']}")
    print()

Column: First column
  Type: float
  Missing: 14.29%
  Unique values: 6

Column: two
  Type: object
  Missing: 14.29%
  Unique values: 6

Column: three
  Type: object
  Missing: 0.0%
  Unique values: 7

Column: SOME;NAME
  Type: integer
  Missing: 0.0%
  Unique values: 4

Column: five
  Type: object
  Missing: 0.0%
  Unique values: 3

Column: What is this even?
  Type: object
  Missing: 0.0%
  Unique values: 3

Column: seven
  Type: object
  Missing: 0.0%
  Unique values: 7

Column: eight
  Type: object
  Missing: 0.0%
  Unique values: 7



In [6]:
# Save column metadata to JSON
with open(OUTPUT_PATH / "column_analysis.json", "w") as json_file:
    json.dump(columns_metadata, json_file, indent=2)

print("Column metadata saved to output/column_analysis.json")

Column metadata saved to output/column_analysis.json


## 4. Statistical Summary Generation

In [ ]:
def generate_statistical_summary(dataframe):
    
    stats = {}
    
    for col in dataframe.columns:
        if pd.api.types.is_numeric_dtype(dataframe[col]):
            # Numerical column statistics
            stats[col] = {
                "type": "numerical",
                "count": int(dataframe[col].count()),
                "mean": float(dataframe[col].mean()),
                "std": float(dataframe[col].std()),
                "min": float(dataframe[col].min()),
                "25%": float(dataframe[col].quantile(0.25)),
                "50%": float(dataframe[col].quantile(0.50)),
                "75%": float(dataframe[col].quantile(0.75)),
                "max": float(dataframe[col].max()),
                "missing": int(dataframe[col].isnull().sum())
            }
        else:
            # Categorical column statistics
            value_counts = dataframe[col].value_counts()
            most_frequent = value_counts.index[0] if len(value_counts) > 0 else None
            
            stats[col] = {
                "type": "categorical",
                "count": int(dataframe[col].count()),
                "unique": int(dataframe[col].nunique()),
                "most_frequent": str(most_frequent) if most_frequent is not None else None,
                "frequency": int(value_counts.iloc[0]) if len(value_counts) > 0 else 0,
                "missing": int(dataframe[col].isnull().sum())
            }
    
    return stats

# Generate statistical summary
statistical_summary = generate_statistical_summary(df)

# Display summary for first few columns
for col_name, stats in list(statistical_summary.items())[:3]:
    print(f"\n{col_name} ({stats['type']})")
    for key, value in stats.items():
        if key != 'type':
            print(f"  {key}: {value}")


First column (numerical)
  count: 6
  mean: -0.5009940002009552
  std: 0.8839385203395562
  min: -1.55529041326908
  25%: -1.247038692513933
  50%: -0.4162433767179556
  75%: 0.1799426841401469
  max: 0.5271122588523375
  missing: 1

two (categorical)
  count: 6
  unique: 6
  most_frequent: -0.14509562920877161
  frequency: 1
  missing: 1

three (categorical)
  count: 7
  unique: 7
  most_frequent: -0.012336991474672475
  frequency: 1
  missing: 0


In [8]:
# Save statistical summary to JSON
with open(OUTPUT_PATH / "statistical_summary.json", "w") as f:
    json.dump(statistical_summary, f, indent=2)

print("Statistical summary saved to output/statistical_summary.json")

Statistical summary saved to output/statistical_summary.json


## 5. Column Name Cleaning and Normalization

In [ ]:
def clean_column_names(dataframe):
    
    df_cleaned = dataframe.copy()
    
    # Clean column names: remove special characters, convert to lowercase, replace spaces with underscores
    cleaned_columns = []
    for col in df_cleaned.columns:
        # Remove special characters except letters, numbers, spaces, and underscores
        cleaned = re.sub(r"[^A-Za-z0-9_ ]", "", str(col))
        # Convert to lowercase and replace spaces with underscores
        cleaned = cleaned.lower().replace(" ", "_").strip()
        # Remove multiple consecutive underscores
        cleaned = re.sub(r"_+", "_", cleaned)
        # Remove leading/trailing underscores
        cleaned = cleaned.strip("_")
        cleaned_columns.append(cleaned)
    
    df_cleaned.columns = cleaned_columns
    return df_cleaned

# Show original column names
print("Original columns:", list(df.columns))

# Clean column names
df_clean_cols = clean_column_names(df)

# Show cleaned column names
print("Cleaned columns:", list(df_clean_cols.columns))

Original columns: ['First column', 'two', 'three', 'SOME;NAME', 'five', 'What is this even?', 'seven', 'eight']
Cleaned columns: ['first_column', 'two', 'three', 'somename', 'five', 'what_is_this_even', 'seven', 'eight']


In [10]:
# Save dataset with cleaned column names
df_clean_cols.to_csv(OUTPUT_PATH / "cleaned_columns_dataset.csv", index=False)
print("Dataset with cleaned columns saved to output/cleaned_columns_dataset.csv")

Dataset with cleaned columns saved to output/cleaned_columns_dataset.csv


## 6. Data Export in Multiple Formats

In [11]:
# Export to various formats
export_formats = {
    'json': lambda df, path: df.to_json(path, orient='records', indent=2),
    'pickle': lambda df, path: df.to_pickle(path),
    'excel': lambda df, path: df.to_excel(path, index=False),
    'parquet': lambda df, path: df.to_parquet(path, index=False)
}

base_filename = "pandas_basics_dataset"

for format_name, export_func in export_formats.items():
    try:
        file_path = OUTPUT_PATH / f"{base_filename}.{format_name}"
        export_func(df_clean_cols, file_path)
        print(f"✓ Exported to {format_name.upper()}: {file_path}")
    except Exception as e:
        print(f"✗ Failed to export to {format_name.upper()}: {e}")

✓ Exported to JSON: output\pandas_basics_dataset.json
✓ Exported to PICKLE: output\pandas_basics_dataset.pickle
✗ Failed to export to EXCEL: No module named 'openpyxl'
✗ Failed to export to PARQUET: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.


## 7. Working with Additional Data Files

In [12]:
# Load and process additional datasets if they exist
additional_files = ['proj1_ex05.pkl', 'proj1_ex06.json']

for filename in additional_files:
    file_path = DATA_PATH / filename
    
    if file_path.exists():
        print(f"\nProcessing {filename}:")
        
        try:
            if filename.endswith('.pkl'):
                data = pd.read_pickle(file_path)
                print(f"  Loaded pickle file with shape: {data.shape}")
                
                # Demonstrate data selection and processing
                if hasattr(data, 'iloc') and data.shape[1] >= 3:
                    selected_columns = data.iloc[:, 1:3]
                    print(f"  Selected columns 1-2, shape: {selected_columns.shape}")
                    
                    # Filter rows based on index pattern if applicable
                    if hasattr(data.index, 'str'):
                        filtered_rows = data[data.index.str.startswith('v', na=False)]
                        if not filtered_rows.empty:
                            print(f"  Filtered rows starting with 'v': {len(filtered_rows)}")
                            
                            # Process and save
                            result_df = selected_columns.loc[filtered_rows.index].fillna('')
                            output_file = OUTPUT_PATH / f"processed_{filename.replace('.pkl', '.csv')}"
                            result_df.to_csv(output_file)
                            print(f"  Saved processed data to: {output_file}")
            
            elif filename.endswith('.json'):
                with open(file_path, 'r') as f:
                    json_data = json.load(f)
                
                # Normalize JSON data
                normalized_df = pd.json_normalize(json_data)
                print(f"  Normalized JSON to DataFrame with shape: {normalized_df.shape}")
                
                # Save normalized data
                output_file = OUTPUT_PATH / f"normalized_{filename.replace('.json', '.csv')}"
                normalized_df.to_csv(output_file, index=False)
                print(f"  Saved normalized data to: {output_file}")
                
        except Exception as e:
            print(f"  Error processing {filename}: {e}")
    else:
        print(f"\n{filename} not found, skipping...")


Processing proj1_ex05.pkl:
  Loaded pickle file with shape: (7, 3)
  Selected columns 1-2, shape: (7, 2)
  Filtered rows starting with 'v': 2
  Saved processed data to: output\processed_proj1_ex05.csv

Processing proj1_ex06.json:
  Normalized JSON to DataFrame with shape: (6, 8)
  Saved normalized data to: output\normalized_proj1_ex06.csv


## 8. Summary and Results

This notebook demonstrated essential pandas operations:

1. **Data Loading**: Successfully loaded CSV data and explored its structure
2. **Column Analysis**: Generated comprehensive metadata for each column
3. **Statistical Summaries**: Created detailed statistics for both numerical and categorical columns
4. **Data Cleaning**: Normalized column names using regex patterns
5. **Multi-format Export**: Saved data in JSON, Pickle, Excel, and Parquet formats
6. **Advanced Processing**: Handled additional data files with filtering and normalization
